In [1]:
import numpy as np
import pandas as pd
import math
import datetime
import os
from scipy import stats
#import statsmodels.formula.api as sm
import sys

In [4]:
result = {}
tickerFile = "sp1500tickerWoSlash.csv"
with open(tickerFile) as csvfile:
    tickers = csvfile.read().split("\n")
i = 0
for ticker in tickers:
    pricepFile = "./sp1500data/sp1500price/"+ticker+"Price.csv"
    sentimentFile = "./sp1500data/sp1500sentiment/"+ticker+"Sentiment.csv"
    priceData = pd.read_csv(pricepFile, sep="\t", skiprows=1,
                        names = ["Date","Ticker","Price"], low_memory=False)
    sentimentData = pd.read_csv(sentimentFile, sep=",", skiprows=1,
                         names = ["UTCTime","Ticker","Sentiment","Date"], low_memory=False)
    if not priceData.empty:
        priceData.Date = priceData.Date.apply(lambda x: pd.to_datetime(x,format='%m/%d/%Y').date())
        priceData = priceData.set_index("Date")
        priceData["Return"] = priceData.Price/priceData.Price.shift(1) - 1
        sentimentData = sentimentData.drop("UTCTime", axis=1)
        sentimentData.Date = sentimentData.Date.apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d').date())
        sentimentData = sentimentData.set_index("Date")
        Score = sentimentData.groupby("Date")["Sentiment"].apply(lambda x: np.log((list(x).count(1)+1)/(list(x).count(-1)+1)))
        result[ticker] = pd.merge(priceData, Score.to_frame(), how = "left", left_index = True, right_index = True)
    i += 1
    sys.stdout.write('\r%.2f%%'%((i+1)*100/len(tickers)))
    sys.stdout.flush()

100.07%

In [5]:
tickersUse = [k for k in result.keys()]
nclass = 5
quantileReturn = {i:[] for i in range(nclass)}
Dates = result[tickersUse[1]].index
for i in range(len(Dates)-1):
    Date = Dates[i]
    scoreList = []
    nextdayReturnList = []
    for ticker in tickersUse:
        if Date in result[ticker].index and Dates[i+1] in result[ticker].index:
            if not np.isnan(result[ticker].loc[Date]["Sentiment"]) and not np.isnan(result[ticker].loc[Dates[i+1]]["Return"]):
                scoreList.append(result[ticker].loc[Date]["Sentiment"])
                nextdayReturnList.append(result[ticker].loc[Dates[i+1]]["Return"])
    returnUse = list(zip(scoreList, nextdayReturnList))
    returnSorted = sorted(returnUse, key = lambda x:x[0])
    n = len(returnSorted)//nclass
    for j in range(nclass):
        quantileReturn[j].append(np.mean([r for s,r in returnSorted[j*n:(j+1)*n]]))
    sys.stdout.write("\r%.2f%%"&((i+1)*100/len(Dates)))

TypeError: unsupported operand type(s) for &: 'str' and 'float'

In [98]:
#每档cummulative return,从分低到分高
[np.cumprod([x+1 for x in v if not x==np.inf])[-1] for k,v in quantileReturn.items()]

[1.4989867884538546,
 3.3346375852439203,
 9.015021503983661,
 7.2652719975885649,
 33.491728420855821]

In [ ]:
stats.describe(newsFactorReturn)

In [ ]:
stats.ttest_1samp(newsFactorReturn,0)